# Face Recognition - Generate Eigenface Classifiers

This notebook tests face detection and recognition from sample images

In [1]:
import cv2, cv2.face
import os, os.path
import numpy as np
from PIL import Image

In [7]:
# constants
CASCADE_PATH = './data/haarcascades/'
CASCADE_NAME = './haarcascade_frontalface_default.xml'
SAMPLE_IMAGES = './data/sample/'

In [3]:
# Viola-Jones classifier for Haar feature extraction 
cascader = cv2.CascadeClassifier(os.path.join(CASCADE_PATH, CASCADE_NAME))
cascader_kwargs = { # setup detection args here
    'scaleFactor': 1.1,
    'minNeighbors': 1,
    'flags': 0
}

In [4]:
# face recognizer
recognizer = cv2.face.createEigenFaceRecognizer()

In [9]:
# function that gets images with respective labels from a given folder
def get_images_and_labels(folder, debug=False, debug_faces=True):
    
    labels = []
    images = []
    
    for label in os.listdir(folder):
        
        if debug: print(label)
        added = 0
        path = os.path.join(folder, label)
        
        if os.path.isdir(path):
            
            for filename in os.listdir(path):
                
                ok = 0

                try:
                    image_path = os.path.join(path, filename)

                    # get grayscale image
                    image_pil = Image.open(image_path).convert('L')
                    image = np.array(image_pil, 'uint8')

                    # get the face using Viola-Jones detector
                    faces = cascader.detectMultiScale(image, **cascader_kwargs)
                    for (x, y, w, h) in faces:
                        images.append(image[y:y+h, x:x+w])
                        labels.append(label)
                        if debug_faces:
                            cv2.imshow('Adding faces to training set...', image[y:y+h, x:x+w])
                            cv2.waitKey(50)
                    
                    ok += len(faces)

                except:
                    pass
                
                if debug: print('\t' + filename + (' [%d face]' % ok))
                if ok: added += 1
        
        if debug: print('[Added %d images]' % added)
    
    if debug_faces:
        cv2.destroyAllWindows()
        
    return images, labels
    

In [11]:
# try getting labels from sample images folder
imlabels = get_images_and_labels(SAMPLE_IMAGES)
imlabels

([array([[180, 141, 109, ...,  52,  44,  45],
         [183, 144, 111, ...,  54,  41,  41],
         [187, 148, 116, ...,  54,  43,  41],
         ..., 
         [105, 102,  99, ...,  57,  58,  59],
         [105, 102,  99, ...,  58,  58,  57],
         [105, 103, 100, ...,  58,  57,  56]], dtype=uint8),
  array([[ 79,  82,  89, ..., 253, 254, 254],
         [ 84,  86,  90, ..., 252, 252, 254],
         [ 89,  90,  93, ..., 251, 251, 253],
         ..., 
         [ 59,  59,  68, ..., 235, 244, 248],
         [ 63,  60,  53, ..., 196, 212, 223],
         [ 63,  71,  62, ..., 134, 147, 162]], dtype=uint8),
  array([[ 57,  69,  73, ...,   1,   1,   1],
         [ 62,  71,  73, ...,   1,   1,   1],
         [ 55,  60,  59, ...,   1,   1,   1],
         ..., 
         [221, 231, 240, ...,  62,  66,  71],
         [221, 225, 230, ...,  39,  41,  42],
         [223, 222, 219, ...,  19,  20,  20]], dtype=uint8),
  array([[152, 158, 167, ...,  71,  80, 101],
         [162, 153, 155, ...,  62,  